In [1]:
! pip install PyPDF2
! pip install chromadb
! pip install tavily
! pip install nemoguardrails

In [26]:
from openai import OpenAI
import gradio as gr
import os
import PyPDF2
import chromadb
from dotenv import load_dotenv
load_dotenv()
import requests
from urllib.parse import urlparse
import re
import json
from datetime import datetime
import uuid

In [3]:
os.environ["OPENAI_API_KEY"] = "" #Add your OPENAI API Key
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key="")

In [28]:
# LangSmith imports with error handling
os.environ["LANGCHAIN_API_KEY"] = ""  #Add your LANGCHAIN API Key
try:
    import langsmith
    from langsmith import traceable
    LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
    if LANGCHAIN_API_KEY:
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_PROJECT"] = "physics-teacher-bot"
        langsmith_client = langsmith.Client()
        LANGCHAIN_AVAILABLE = True
        print("✅ LangSmith tracing initialized successfully!")
    else:
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        langsmith_client = None
        LANGCHAIN_AVAILABLE = False
except ImportError:
    print("⚠️  LangSmith not installed. Tracing will be disabled.")
    print("💡 Run: pip install langsmith")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False
except Exception as e:
    print(f"⚠️  LangSmith initialization failed: {e}")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False

✅ LangSmith tracing initialized successfully!


In [29]:
# Initialize Tavily client with error handling
os.environ["TAVILY_API_KEY"] = ""  #Add your TAVILY API Key
try:
    from tavily import TavilyClient
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Web search functionality will be disabled.")
    print("💡 Run: pip install tavily-python")

In [30]:
# Initialize NVIDIA NeMo Guardrails with error handling
try:
    from nemoguardrails import RailsConfig, LLMRails
    from nemoguardrails.actions import action
    NEMO_GUARDRAILS_AVAILABLE = True
except ImportError:
    NEMO_GUARDRAILS_AVAILABLE = False
    print("⚠️  NVIDIA NeMo Guardrails not installed. Safety features will be disabled.")
    print("💡 Run: pip install nemoguardrails")

In [6]:
# Initialize ChromaDB
chroma_client = chromadb.Client()
pdf_collection = chroma_client.create_collection(name="PhysicsPDFs")
memory_collection = chroma_client.create_collection(name="ConversationMemory")

In [31]:
# Initialize NeMo Guardrails if available
nemo_rails = None
if NEMO_GUARDRAILS_AVAILABLE:
    try:
        # Define complete guardrails configuration
        guardrails_config = """
models:
  - type: main
    engine: openai
    model: gpt-3.5-turbo

rails:
  input:
    flows:
      - self check input
      - check jailbreak
      - detect sensitive topics
  output:
    flows:
      - self check output
      - check for pii
      - ensure appropriate tone

instructions:
  - type: general
    content: |
      You are a physics teaching assistant. Always provide accurate, educational,
      and safe responses. Focus on physics concepts and related scientific topics.
      Avoid harmful, dangerous, or inappropriate content.

prompts:
  - task: self_check_input
    content: |
      Check if the following user message is safe and appropriate for a physics education context.
      Consider if it contains any harmful, dangerous, illegal, or inappropriate content.
      Focus on educational physics topics.

      User message: {{user_input}}

      Is this message safe and appropriate? Answer with YES or NO only.

  - task: self_check_output
    content: |
      Check if the following bot response is safe, appropriate, and follows the guidelines for a physics education context.
      The response should be educational, accurate, and avoid any harmful or inappropriate content.

      Bot response: {{bot_response}}

      Is this response safe and appropriate? Answer with YES or NO only.

  - task: general
    content: |
      You are a physics teaching assistant. Provide accurate, educational, and safe responses.
      Focus on physics concepts and related scientific topics.
      If asked about harmful, dangerous, or inappropriate topics, politely decline and redirect to educational physics topics.
        """

        config = RailsConfig.from_content(guardrails_config)
        nemo_rails = LLMRails(config)

        # Register custom actions
        @action
        def check_physics_context():
            return True

        @action
        def validate_educational_content():
            return True

        nemo_rails.register_action(check_physics_context)
        nemo_rails.register_action(validate_educational_content)

        print("✅ NVIDIA NeMo Guardrails initialized successfully!")

    except Exception as e:
        print(f"❌ Error initializing NeMo Guardrails: {e}")
        NEMO_GUARDRAILS_AVAILABLE = False

✅ NVIDIA NeMo Guardrails initialized successfully!


In [32]:
class TokenUsageTracker:
    """Track token usage across different components"""

    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.costs = 0.0

    def update_from_response(self, response, model="gpt-3.5-turbo"):
        """Update token counts from OpenAI response"""
        if hasattr(response, 'usage'):
            usage = response.usage
            self.prompt_tokens += getattr(usage, 'prompt_tokens', 0)
            self.completion_tokens += getattr(usage, 'completion_tokens', 0)
            self.total_tokens += getattr(usage, 'total_tokens', 0)

            # Calculate cost (approximate)
            cost_rates = {
                "gpt-3.5-turbo": (0.0015, 0.002),  # $0.0015/1K input, $0.002/1K output
                "gpt-4": (0.03, 0.06),  # $0.03/1K input, $0.06/1K output
                "gpt-4-turbo": (0.01, 0.03)  # $0.01/1K input, $0.03/1K output
            }

            input_rate, output_rate = cost_rates.get(model, (0.0015, 0.002))
            self.costs += (self.prompt_tokens / 1000 * input_rate) + (self.completion_tokens / 1000 * output_rate)

    def get_summary(self):
        """Get token usage summary"""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "estimated_cost": round(self.costs, 4)
        }

In [33]:
# Global token tracker
token_tracker = TokenUsageTracker()

def traceable_function(func):
    """Decorator that conditionally applies tracing based on availability"""
    if LANGCHAIN_AVAILABLE:
        return traceable(func)
    return func

In [35]:
@traceable_function
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [37]:
@traceable_function
def chunk_text(text, chunk_size=1000):
    """Split text into chunks"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [38]:
@traceable_function
def store_pdf_in_db(pdf_path):
    """Extract, chunk and store PDF in ChromaDB"""
    try:
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)

        # Clear existing data
        pdf_collection.delete(where={"source": {"$ne": ""}})

        # Add chunks to ChromaDB
        for i, chunk in enumerate(chunks):
            pdf_collection.add(
                documents=[chunk],
                metadatas=[{"source": pdf_path, "chunk_id": i, "type": "pdf"}],
                ids=[f"pdf_chunk_{i}_{os.path.basename(pdf_path)}"]
            )

        # Log to LangSmith
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,  # Will be associated with current trace
                    key="pdf_processing",
                    score=1.0,
                    comment=f"Processed {len(chunks)} chunks from {pdf_path}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Successfully loaded {len(chunks)} chunks from {pdf_path}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [39]:
@traceable_function
def get_relevant_context(query, n_results=3):
    """Retrieve relevant context from PDF knowledge base"""
    try:
        results = pdf_collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return "\n\n".join(results['documents'][0]) if results['documents'] else ""
    except Exception as e:
        print(f"PDF context query error: {e}")
        return ""

In [40]:
@traceable_function
def store_conversation_memory(user_message, bot_response, conversation_context=""):
    """Store conversation in long-term memory"""
    try:
        # Create a memory entry
        memory_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()

        # Combine user message and bot response for semantic search
        memory_text = f"User: {user_message}\nBot: {bot_response}"

        if conversation_context:
            memory_text = f"Context: {conversation_context}\n{memory_text}"

        # Store in memory collection
        memory_collection.add(
            documents=[memory_text],
            metadatas=[{
                "type": "conversation",
                "timestamp": timestamp,
                "user_message": user_message,
                "bot_response": bot_response,
                "conversation_context": conversation_context
            }],
            ids=[memory_id]
        )

        return True
    except Exception as e:
        print(f"Memory storage error: {e}")
        return False

In [41]:
@traceable_function
def get_relevant_memories(query, n_results=2):
    """Retrieve relevant past conversations from memory"""
    try:
        results = memory_collection.query(
            query_texts=[query],
            n_results=n_results
        )

        memories = []
        if results['documents']:
            for i, doc in enumerate(results['documents'][0]):
                metadata = results['metadatas'][0][i]
                memory_text = f"Previous conversation ({metadata['timestamp'][:10]}): {doc}"
                memories.append(memory_text)

        return "\n\n".join(memories) if memories else ""
    except Exception as e:
        print(f"Memory retrieval error: {e}")
        return ""

In [44]:
@traceable_function
def get_conversation_summary(history):
    """Generate a summary of the current conversation context"""
    if not history:
        return ""

    # Take last few exchanges for context
    recent_history = history[-3:] if len(history) > 3 else history
    context_parts = []

    for exchange in recent_history:
        if isinstance(exchange, tuple) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")
        elif isinstance(exchange, list) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")

    return "\n".join(context_parts)

In [45]:
@traceable_function
def is_pdf_context_relevant(pdf_context, query):
    """Check if PDF context is actually relevant to the query"""
    if not pdf_context or len(pdf_context.strip()) < 50:
        return False

    # Check for common irrelevant patterns in PDF context
    irrelevant_patterns = [
        "table of contents", "copyright", "abstract", "references",
        "appendix", "index", "acknowledg", "preface"
    ]

    pdf_lower = pdf_context.lower()
    query_lower = query.lower()

    # If PDF context contains many irrelevant sections, it's likely not useful
    irrelevant_count = sum(1 for pattern in irrelevant_patterns if pattern in pdf_lower)
    if irrelevant_count > 2:
        return False

    # Check if query terms actually appear in the context
    query_terms = query_lower.split()
    relevant_terms = sum(1 for term in query_terms if len(term) > 3 and term in pdf_lower)

    # If less than 30% of meaningful query terms are in context, consider it irrelevant
    meaningful_terms = [term for term in query_terms if len(term) > 3]
    if meaningful_terms and (relevant_terms / len(meaningful_terms)) < 0.3:
        return False

    return True

In [46]:
@traceable_function
def search_with_tavily(query):
    """Search using Tavily and get content from first result"""
    if not TAVILY_AVAILABLE:
        return "Web search unavailable. Please install tavily-python: pip install tavily-python"

    try:
        # Search for the query
        search_response = tavily_client.search(
            query=query,
            search_depth="basic",
            max_results=3
        )

        if not search_response.get('results'):
            return "No search results found."

        # Get the first result
        first_result = search_response['results'][0]
        first_url = first_result.get('url', '')

        # Use Tavily's content if available
        content = first_result.get('content', '')

        if not content:
            content = f"Title: {first_result.get('title', 'N/A')}\nURL: {first_url}"

        # Check if it's a PDF
        if first_url.lower().endswith('.pdf'):
            return f"📄 PDF Source: {first_url}\n\nThis appears to be a PDF document. Please visit the link to view the PDF: {first_url}"

        return f"🔍 Search Result from: {first_url}\n\n{content}"

    except Exception as e:
        return f"❌ Search error: {str(e)}"

In [48]:
def apply_keyword_safety_check(user_message, bot_response):
    """Fallback keyword-based safety check"""
    harmful_keywords = [
        "bomb", "weapon", "explosive", "harmful", "dangerous", "illegal",
        "hack", "cheat", "bypass", "malicious", "violence", "hurt", "kill"
    ]

    user_lower = user_message.lower()
    response_lower = bot_response.lower()

    # Check for harmful content in user message
    if any(keyword in user_lower for keyword in harmful_keywords):
        safety_message = "I apologize, but I cannot engage with that type of question. I'm here to help with physics education and related scientific topics."
        return safety_message, "🛡️ Input blocked by safety check"

    # Check for harmful content in bot response
    if any(keyword in response_lower for keyword in harmful_keywords):
        safety_message = "I need to provide a safe and educational response. Let me rephrase that to focus on the educational aspects of physics."
        return safety_message, "🛡️ Response moderated by safety check"

    return bot_response, "🛡️ Safety check passed"

In [47]:
@traceable_function
def apply_safety_guardrails(user_message, bot_response):
    """Apply NeMo Guardrails for safety and content filtering"""
    if not NEMO_GUARDRAILS_AVAILABLE:
        return bot_response, "⚠️ Guardrails disabled"

    try:
        # Use NeMo Guardrails for safety checking
        guarded_response = nemo_rails.generate(
            messages=[{"role": "user", "content": user_message}]
        )

        # If guardrails modified the response, use the guarded version
        if guarded_response and guarded_response != bot_response:
            return guarded_response, "🛡️ Response moderated by guardrails"

        return bot_response, "🛡️ Safety check passed"

    except Exception as e:
        print(f"Guardrails error: {e}")
        # Fallback to keyword-based safety check
        return apply_keyword_safety_check(user_message, bot_response)

In [49]:
@traceable_function
def check_educational_context(user_message):
    """Check if the query is appropriate for educational physics context"""
    inappropriate_topics = [
        "harmful", "dangerous", "illegal", "weapon", "explosive",
        "hack", "cheat", "bypass", "circumvent", "malicious"
    ]

    user_lower = user_message.lower()

    # Check for potentially harmful topics
    for topic in inappropriate_topics:
        if topic in user_lower:
            return False, f"This appears to be asking about {topic} topics, which I cannot assist with for safety reasons."

    # Check if it's physics-related or general educational
    physics_terms = [
        "physics", "quantum", "mechanics", "thermodynamics", "electromagnetism",
        "relativity", "optics", "atomic", "nuclear", "particle", "force",
        "energy", "motion", "velocity", "acceleration", "gravity", "electric",
        "magnetic", "wave", "light", "sound", "heat", "temperature"
    ]

    # Allow general educational questions too
    educational_terms = [
        "science", "math", "chemistry", "biology", "astronomy", "engineering",
        "technology", "computer", "programming", "learn", "study", "education",
        "explain", "how does", "what is", "why does"
    ]

    has_physics_term = any(term in user_lower for term in physics_terms)
    has_educational_term = any(term in user_lower for term in educational_terms)

    if not (has_physics_term or has_educational_term):
        return False, "I specialize in physics and educational topics. Please ask questions related to science, education, or general knowledge."

    return True, "OK"

In [50]:
@traceable_function
def chat_with_physics_bot(message, history):
    """Main chat function with comprehensive tracing and token tracking"""

    # Create a unique run ID for this conversation
    run_id = str(uuid.uuid4())

    # Log conversation start
    if langsmith_client:
        try:
            langsmith_client.create_feedback(
                run_id=run_id,
                key="conversation_start",
                score=1.0,
                comment=f"User message: {message[:100]}..."
            )
        except Exception as e:
            print(f"LangSmith logging error: {e}")

    # First check educational context
    is_appropriate, context_message = check_educational_context(message)
    if not is_appropriate:
        return f"🛡️ Educational Context Check:\n\n{context_message}"

    # Get relevant context from different sources
    pdf_context = get_relevant_context(message)
    memory_context = get_relevant_memories(message)
    conversation_context = get_conversation_summary(history)

    # Check if PDF context is actually relevant
    use_pdf = is_pdf_context_relevant(pdf_context, message)

    system_prompt = """You are a friendly Physics Teacher Bot with memory. Follow these rules:
    1. Use the provided context from PDFs, past conversations, and search results to answer questions accurately
    2. If the context doesn't fully answer the question, use your knowledge to provide a complete answer
    3. Reference past conversations when relevant to provide continuity
    4. Be educational and clear in your explanations
    5. Maintain context from the current conversation flow
    6. Always prioritize safety and educational value in your responses"""

    # Build context sections
    context_sections = []
    source_notes = []

    if use_pdf:
        context_sections.append(f"PDF Knowledge:\n{pdf_context}")
        source_notes.append("📚 PDF")

    if memory_context:
        context_sections.append(f"Past Conversations:\n{memory_context}")
        source_notes.append("💭 Memory")

    if conversation_context:
        context_sections.append(f"Current Conversation:\n{conversation_context}")

    # If no PDF context is relevant, use web search
    if not use_pdf:
        search_results = search_with_tavily(message)
        context_sections.append(f"Web Search:\n{search_results}")
        source_notes.append("🔍 Web")

    # Combine all contexts
    combined_context = "\n\n".join(context_sections) if context_sections else "No specific context available."
    source_note = " + ".join(source_notes) if source_notes else "General knowledge"

    user_content = f"""Available Context:
{combined_context}

Question: {message}

Please provide a helpful, educational answer based on the available context above."""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ],
            temperature=0.2
        )

        # Track token usage
        token_tracker.update_from_response(response, "gpt-3.5-turbo")

        bot_response = response.choices[0].message.content

        # Apply safety guardrails
        safe_response, safety_status = apply_safety_guardrails(message, bot_response)

        # Store this conversation in long-term memory (only if safe)
        if "blocked" not in safety_status.lower():
            store_conversation_memory(message, safe_response, conversation_context)

        # Log token usage and success
        if langsmith_client:
            try:
                token_summary = token_tracker.get_summary()
                langsmith_client.create_feedback(
                    run_id=run_id,
                    key="token_usage",
                    score=1.0,
                    comment=json.dumps(token_summary)
                )

                langsmith_client.create_feedback(
                    run_id=run_id,
                    key="response_quality",
                    score=1.0,
                    comment=f"Sources: {source_note}, Safety: {safety_status}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        # Add token usage to response
        token_summary = token_tracker.get_summary()
        token_info = f" | Tokens: {token_summary['total_tokens']} (${token_summary['estimated_cost']})"

        return f"🔮 Sources: {source_note} | {safety_status}{token_info}\n\n{safe_response}"

    except Exception as e:
        # Log error to LangSmith
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=run_id,
                    key="error",
                    score=0.0,
                    comment=f"Error generating response: {str(e)}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"❌ Error generating response: {str(e)}"

In [51]:
@traceable_function
def clear_memory():
    """Clear all conversation memory"""
    try:
        memory_collection.delete(where={"type": {"$ne": ""}})

        # Log memory clearance
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_cleared",
                    score=1.0,
                    comment="All conversation memory cleared"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return "✅ Conversation memory cleared successfully!"
    except Exception as e:
        return f"❌ Error clearing memory: {str(e)}"

In [52]:
@traceable_function
def export_memory():
    """Export conversation memory as JSON"""
    try:
        results = memory_collection.get()
        memories = []

        for i, doc in enumerate(results['documents']):
            memories.append({
                "content": doc,
                "metadata": results['metadatas'][i],
                "id": results['ids'][i]
            })

        filename = f"conversation_memory_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(memories, f, indent=2)

        # Log export
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_exported",
                    score=1.0,
                    comment=f"Exported {len(memories)} memories to {filename}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Memory exported to {filename}"
    except Exception as e:
        return f"❌ Error exporting memory: {str(e)}"

In [55]:
def get_guardrails_status():
    """Get status of safety guardrails"""
    if NEMO_GUARDRAILS_AVAILABLE and nemo_rails is not None:
        return "✅ NVIDIA NeMo Guardrails: ACTIVE"
    else:
        return "⚠️ NVIDIA NeMo Guardrails: NOT AVAILABLE"

def get_token_usage():
    """Get current token usage statistics"""
    summary = token_tracker.get_summary()
    return f"""
📊 Token Usage Summary:
• Total Tokens: {summary['total_tokens']:,}
• Prompt Tokens: {summary['prompt_tokens']:,}
• Completion Tokens: {summary['completion_tokens']:,}
• Estimated Cost: ${summary['estimated_cost']}
"""

In [56]:
def reset_token_tracking():
    """Reset token tracking counters"""
    global token_tracker
    token_tracker = TokenUsageTracker()
    return "✅ Token tracking reset successfully!"

In [57]:
def get_langsmith_status():
    """Get LangSmith tracing status"""
    if LANGCHAIN_AVAILABLE:
        return "✅ LangSmith Tracing: ACTIVE"
    else:
        return "⚠️ LangSmith Tracing: NOT AVAILABLE"

In [50]:
def load_pdf_and_chat(pdf_file, message, history):
    # First store the PDF
    result_msg = store_pdf_in_db(pdf_file.name)

    # Then get response
    response = chat_with_physics_bot(message, history)

    return result_msg, response

In [58]:
def launch_app():
    with gr.Blocks() as demo:
        gr.Markdown("# ⚛️ Physics Teacher Chat with PDF RAG + Web Search + Memory + Safety Guardrails + LangSmith Tracing")

        with gr.Row():
            with gr.Column():
                pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
                load_btn = gr.Button("Load PDF into Knowledge Base")
                load_status = gr.Textbox(label="Load Status", interactive=False)

                gr.Markdown("### Safety & Memory Management")
                with gr.Row():
                    clear_mem_btn = gr.Button("🧹 Clear Memory")
                    export_mem_btn = gr.Button("💾 Export Memory")
                    safety_status_btn = gr.Button("🛡️ Safety Status")
                mem_status = gr.Textbox(label="Memory Status", interactive=False)

                gr.Markdown("### Token Usage Tracking")
                with gr.Row():
                    token_stats_btn = gr.Button("📊 Token Usage")
                    reset_tokens_btn = gr.Button("🔄 Reset Tracking")
                    tracing_status_btn = gr.Button("🔍 Tracing Status")
                token_status = gr.Textbox(label="Token Status", interactive=False)

                gr.Markdown("""
                ### How it works:
                1. **Upload a PDF** - Answers prioritize PDF content when relevant
                2. **Ask questions** - System uses PDF, web search, and past conversations
                3. **Long-term memory** - Remembers past conversations for context
                4. **Safety Guardrails** - NVIDIA NeMo Guardrails ensure safe, educational responses
                5. **Multiple sources** - Combines PDF 📚, Web 🔍, and Memory 💭
                6. **LangSmith Tracing** - Comprehensive tracing and token usage tracking
                7. **Token Monitoring** - Real-time token usage and cost tracking
                """)

            with gr.Column():
                chat_interface = gr.ChatInterface(
                    fn=chat_with_physics_bot,
                    title="Chat with Physics Bot (with Memory, Safety & Tracing)",
                    description="Ask questions about physics. I'll remember our past conversations, use multiple knowledge sources, ensure safe responses, and track token usage!"
                )

        # Load PDF when button clicked
        load_btn.click(
            fn=lambda pdf: store_pdf_in_db(pdf.name) if pdf else "Please upload a PDF first",
            inputs=[pdf_input],
            outputs=[load_status]
        )

        # Memory management buttons
        clear_mem_btn.click(
            fn=clear_memory,
            outputs=[mem_status]
        )

        export_mem_btn.click(
            fn=export_memory,
            outputs=[mem_status]
        )

        safety_status_btn.click(
            fn=get_guardrails_status,
            outputs=[mem_status]
        )

        # Token tracking buttons
        token_stats_btn.click(
            fn=get_token_usage,
            outputs=[token_status]
        )

        reset_tokens_btn.click(
            fn=reset_token_tracking,
            outputs=[token_status]
        )

        tracing_status_btn.click(
            fn=get_langsmith_status,
            outputs=[token_status]
        )

    demo.launch()

if __name__ == "__main__":
    # Check for required API keys
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY not found in environment variables")
        exit(1)

    if not os.getenv("TAVILY_API_KEY") and TAVILY_AVAILABLE:
        print("❌ TAVILY_API_KEY not found in environment variables")
        print("Please get your free API key from: https://app.tavily.com/")
        print("Continuing without web search functionality...")

    # Check for LangSmith API key
    if not os.getenv("LANGCHAIN_API_KEY"):
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        print("💡 Get your free API key from: https://smith.langchain.com/")

    # Auto-load PDF if exists
    pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
    if pdf_files:
        print(f"Found PDF: {pdf_files[0]}")
        store_pdf_in_db(pdf_files[0])

    print("🧠 Long-term memory enabled!")
    print("📊 Token usage tracking enabled!")
    print(get_guardrails_status())
    print(get_langsmith_status())

    if LANGCHAIN_AVAILABLE:
        print("🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.")

    launch_app()

🧠 Long-term memory enabled!
📊 Token usage tracking enabled!
✅ NVIDIA NeMo Guardrails: ACTIVE
✅ LangSmith Tracing: ACTIVE
🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78dc615efcbf066159.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
